In [ ]:
import os
import glob
import logging
import numpy as np
import pydicom
import matplotlib.pyplot as plt
from tqdm import tqdm

try:
    import SimpleITK as sitk
    SITK_AVAILABLE = True
except ImportError:
    SITK_AVAILABLE = False
    print("⚠️ SimpleITK is required for this script. Please install it: pip install SimpleITK")

# ตั้งค่าการแสดงผล Log
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_ct_series(folder_path: str) -> list:
    """Loads and sorts a CT series from a folder."""
    ct_slices = []
    logging.info(f"🔍 Searching for CT slices in: {folder_path}")
    # ค้นหาไฟล์ .dcm ในทุกโฟลเดอร์ย่อย
    for root, _, files in os.walk(folder_path):
        for f in files:
            if f.endswith('.dcm'):
                try:
                    dcm_path = os.path.join(root, f)
                    dcm = pydicom.dcmread(dcm_path, force=True)
                    if hasattr(dcm, 'Modality') and dcm.Modality == 'CT':
                        ct_slices.append(dcm)
                except Exception:
                    continue
    
    if not ct_slices:
        return []
        
    ct_slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    logging.info(f"✅ Found and sorted {len(ct_slices)} CT slices.")
    return ct_slices

def find_dose_files(folder_path: str, keyword1: str, keyword2: str) -> tuple:
    """Finds two RTDOSE files based on keywords in filename or DICOM tags."""
    dose_file1, dose_file2 = None, None
    logging.info(f"🔍 Searching RTDOSE files using keywords: '{keyword1}' and '{keyword2}'")
    
    # ค้นหาไฟล์ .dcm ในทุกโฟลเดอร์ย่อย
    for root, _, files in os.walk(folder_path):
        for f in files:
            if f.endswith('.dcm'):
                try:
                    dcm_path = os.path.join(root, f)
                    dcm = pydicom.dcmread(dcm_path, force=True)
                    if hasattr(dcm, 'Modality') and dcm.Modality == 'RTDOSE':
                        fname_lower = os.path.basename(f).lower()
                        series_desc = str(getattr(dcm, 'SeriesDescription', '')).lower()
                        plan_label = str(getattr(dcm, 'RTPlanLabel', '')).lower()
                        
                        # Check for first dose file
                        if (dose_file1 is None and (keyword1.lower() in fname_lower or 
                            keyword1.lower() in series_desc or 
                            keyword1.lower() in plan_label)):
                            dose_file1 = dcm
                            logging.info(f"  -> Found Dose 1 ('{keyword1}'): {os.path.basename(f)}")
                        
                        # Check for second dose file
                        elif (dose_file2 is None and (keyword2.lower() in fname_lower or
                              keyword2.lower() in series_desc or
                              keyword2.lower() in plan_label)):
                            dose_file2 = dcm
                            logging.info(f"  -> Found Dose 2 ('{keyword2}'): {os.path.basename(f)}")
                except Exception:
                    continue
            
    return dose_file1, dose_file2

def align_dose_to_ct(dose_array: np.ndarray, dose_dcm: pydicom.Dataset, ct_slices: list) -> np.ndarray:
    """Aligns a dose grid to the CT grid using SimpleITK."""
    if not SITK_AVAILABLE:
        raise ImportError("SimpleITK is not installed.")
        
    logging.info(f"🔄 Aligning dose '{getattr(dose_dcm, 'RTPlanLabel', 'Unknown')}' to CT grid...")

    # สร้าง CT reference image จากข้อมูล DICOM
    reader = sitk.ImageSeriesReader()
    dicom_names = [s.filename for s in ct_slices]
    reader.SetFileNames(dicom_names)
    ref_image = reader.Execute()
    logging.info(f"  [CT Ref] Origin: {ref_image.GetOrigin()}, Spacing: {ref_image.GetSpacing()}, Size: {ref_image.GetSize()}")
    
    # สร้าง Dose image
    # pydicom อ่าน dose array เป็น (z, y, x) ซึ่ง sitk.GetImageFromArray เข้าใจอยู่แล้ว
    dose_sitk = sitk.GetImageFromArray(dose_array) 
    dose_sitk.SetOrigin(np.array(dose_dcm.ImagePositionPatient, dtype=np.float64))
    dose_z_spacing = abs(dose_dcm.GridFrameOffsetVector[1] - dose_dcm.GridFrameOffsetVector[0]) if len(dose_dcm.GridFrameOffsetVector) > 1 else 3.0
    dose_spacing = np.array([dose_dcm.PixelSpacing[0], dose_dcm.PixelSpacing[1], dose_z_spacing], dtype=np.float64)
    dose_sitk.SetSpacing(dose_spacing)
    
    # --- FIX for SetDirection ---
    iop = np.array(dose_dcm.ImageOrientationPatient, dtype=np.float64)
    row_cosine = iop[0:3]
    col_cosine = iop[3:6]
    slice_cosine = np.cross(row_cosine, col_cosine)
    direction_matrix = np.stack((row_cosine, col_cosine, slice_cosine)).ravel()
    dose_sitk.SetDirection(direction_matrix)
    logging.info(f"  [Dose] Origin: {dose_sitk.GetOrigin()}, Spacing: {dose_sitk.GetSpacing()}, Size: {dose_sitk.GetSize()}")

    # Resample dose to match CT grid
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ref_image)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(0.0)
    aligned_dose_sitk = resampler.Execute(dose_sitk)
    
    # แปลงกลับเป็น numpy array (z, y, x)
    aligned_dose_np = sitk.GetArrayFromImage(aligned_dose_sitk)
    logging.info(f"  -> Alignment successful. Output shape: {aligned_dose_np.shape}")
    return aligned_dose_np

def visualize_and_save_all_slices(output_folder: str, ct_vol: np.ndarray, dose1_vol: np.ndarray, dose2_vol: np.ndarray, diff_vol: np.ndarray, dose1_label: str, dose2_label: str):
    """Visualizes and saves all slices as high-resolution PNG files."""
    
    os.makedirs(output_folder, exist_ok=True)
    logging.info(f"💾 Saving all slices to: {output_folder}")

    # CT Windowing for better contrast
    win_center, win_width = 40, 400
    vmin_ct, vmax_ct = win_center - win_width / 2, win_center + win_width / 2
    
    # Determine consistent color scales
    vmax_d1 = np.percentile(dose1_vol, 99.9) if np.max(dose1_vol) > 0 else 1.0
    vmax_d2 = np.percentile(dose2_vol, 99.9) if np.max(dose2_vol) > 0 else 1.0
    diff_max_abs = np.percentile(np.abs(diff_vol), 99.9) if np.max(np.abs(diff_vol)) > 0 else 1.0

    for slice_idx in tqdm(range(ct_vol.shape[0]), desc="Saving Slices"):
        ct_slice = ct_vol[slice_idx]
        dose1_slice = dose1_vol[slice_idx]
        dose2_slice = dose2_vol[slice_idx]
        diff_slice = diff_vol[slice_idx]

        fig, axs = plt.subplots(1, 4, figsize=(22, 6))
        fig.suptitle(f'Comparison - Slice {slice_idx}', fontsize=16)

        # Panel 1: CT Only
        axs[0].imshow(ct_slice, cmap='gray', vmin=vmin_ct, vmax=vmax_ct)
        axs[0].set_title(f'CT image')
        axs[0].axis('off')

        # Panel 2: CT + GT Dose
        axs[1].imshow(ct_slice, cmap='gray', vmin=vmin_ct, vmax=vmax_ct)
        im2 = axs[1].imshow(dose1_slice, cmap='jet', alpha=0.5, vmin=0, vmax=vmax_d1)
        axs[1].set_title(f'Ground truth')
        axs[1].axis('off')
        fig.colorbar(im2, ax=axs[1], label='Dose (Gy)', fraction=0.046, pad=0.04)

        # Panel 3: CT + PD Dose
        axs[2].imshow(ct_slice, cmap='gray', vmin=vmin_ct, vmax=vmax_ct)
        im3 = axs[2].imshow(dose2_slice, cmap='jet', alpha=0.5, vmin=0, vmax=vmax_d2)
        axs[2].set_title(f'Prediction')
        axs[2].axis('off')
        fig.colorbar(im3, ax=axs[2], label='Dose (Gy)', fraction=0.046, pad=0.04)

        # Panel 4: Difference Overlay
        axs[3].imshow(ct_slice, cmap='gray', vmin=vmin_ct, vmax=vmax_ct)
        im4 = axs[3].imshow(diff_slice, cmap='bwr', alpha=0.6, vmin=-diff_max_abs, vmax=diff_max_abs)
        axs[3].set_title(f'Dose Difference')
        axs[3].axis('off')
        fig.colorbar(im4, ax=axs[3], label='Dose Difference (Gy)', fraction=0.046, pad=0.04)

        plt.tight_layout(rect=[0, 0, 1, 0.95])
        
        # Save the figure with high resolution
        save_path = os.path.join(output_folder, f'slice_{slice_idx:03d}.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close(fig) # Close the figure to free up memory

def main():
    """Main workflow to load, process, and visualize the dose comparison."""
    
    # --- ‼️‼️ แก้ไข 3 ค่านี้ให้ตรงกับข้อมูลของคุณ ‼️‼️ ---
    # 1. ที่อยู่ของโฟลเดอร์ที่มีไฟล์ DICOM ทั้งหมด
    DICOM_FOLDER = r"d:\Workhard\drive-download-20250713T211807Z-1-001\Patient45_ForImport"

    # 2. คีย์เวิร์ดสำหรับระบุไฟล์ RTDOSE ชุดที่ 1
    DOSE_1_KEYWORD = "1.2.246.352.221.4881137948538846801.5291334637160488349" # เช่น 'gt', 'plan1', 'brachy'

    # 3. คีย์เวิร์ดสำหรับระบุไฟล์ RTDOSE ชุดที่ 2
    DOSE_2_KEYWORD = "OVERWRITE" # เช่น 'pd', 'final', 'approved'
    # ----------------------------------------------------
    
    # --- ตั้งค่าโฟลเดอร์สำหรับเก็บผลลัพธ์ ---
    PATIENT_ID = os.path.basename(DICOM_FOLDER)
    OUTPUT_FOLDER = os.path.join(os.path.dirname(DICOM_FOLDER), "results", PATIENT_ID)

    if not os.path.isdir(DICOM_FOLDER) or not SITK_AVAILABLE:
        if not SITK_AVAILABLE:
            logging.error("❌ SimpleITK is not available. Please install it.")
        else:
            logging.error(f"❌ Directory not found: {DICOM_FOLDER}")
        return

    # 1. Load data
    ct_slices = load_ct_series(DICOM_FOLDER)
    dose1_dcm, dose2_dcm = find_dose_files(DICOM_FOLDER, DOSE_1_KEYWORD, DOSE_2_KEYWORD)

    if not ct_slices or not dose1_dcm or not dose2_dcm:
        logging.error("❌ Failed to load required files (CT and/or both RTDOSE). Please check paths and keywords.")
        return

    # 2. Process into numpy arrays
    ct_volume = np.stack([s.pixel_array * float(getattr(s, 'RescaleSlope', 1.0)) + float(getattr(s, 'RescaleIntercept', 0.0)) for s in ct_slices], axis=0)
    dose1_raw = dose1_dcm.pixel_array.astype(np.float32) * float(dose1_dcm.DoseGridScaling)
    dose2_raw = dose2_dcm.pixel_array.astype(np.float32) * float(dose2_dcm.DoseGridScaling)

    # 3. Align doses to CT
    try:
        aligned_dose1 = align_dose_to_ct(dose1_raw, dose1_dcm, ct_slices)
        aligned_dose2 = align_dose_to_ct(dose2_raw, dose2_dcm, ct_slices)
    except Exception as e:
        logging.error(f"❌ Dose alignment failed: {e}")
        return
        
    # 4. Calculate difference
    difference_volume = aligned_dose1 - aligned_dose2
    
    # 5. Visualize and Save All Slices
    visualize_and_save_all_slices(
        output_folder=OUTPUT_FOLDER,
        ct_vol=ct_volume, 
        dose1_vol=aligned_dose1, 
        dose2_vol=aligned_dose2, 
        diff_vol=difference_volume, 
        dose1_label=DOSE_1_KEYWORD, 
        dose2_label=DOSE_2_KEYWORD
    )
    
    logging.info("🎉 Visualization complete!")


if __name__ == '__main__':
    main()

2025-07-14 06:21:57,532 - INFO - 🔍 Searching for CT slices in: d:\Workhard\drive-download-20250713T211807Z-1-001\Patient45_ForImport
2025-07-14 06:22:11,255 - INFO - ✅ Found and sorted 166 CT slices.
2025-07-14 06:22:11,258 - INFO - 🔍 Searching RTDOSE files using keywords: '1.2.246.352.221.4881137948538846801.5291334637160488349' and 'OVERWRITE'
2025-07-14 06:22:11,448 - INFO -   -> Found Dose 1 ('1.2.246.352.221.4881137948538846801.5291334637160488349'): RD.1.2.246.352.221.4881137948538846801.5291334637160488349.dcm
2025-07-14 06:22:11,461 - INFO -   -> Found Dose 2 ('OVERWRITE'): RD.OVERWRITE.BT045_postprocessed_SAFE.dcm
2025-07-14 06:22:11,878 - INFO - 🔄 Aligning dose 'Unknown' to CT grid...
2025-07-14 06:22:12,402 - INFO -   [CT Ref] Origin: (-246.0, -140.0, -1331.5), Spacing: (0.9609375, 0.9609375, 2.0), Size: (512, 512, 166)
2025-07-14 06:22:12,407 - INFO -   [Dose] Origin: (-199.91911764706, -16.928275997899, -1331.5), Spacing: (2.5, 2.5, 2.0), Size: (166, 98, 166)
2025-07-14 06